# Modelo BIOS:

## Importacion de Librerias

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import pulp as pu
from problema.asignador_capacidad import AsignadorCapacidad
import os

## Parametros generales

In [ ]:
file = '0_model_template_0131_v2.xlsm'

# Capacidad de carga de un camion
cap_camion = 34000

# Capacidad de descarga en puerto por día
cap_descarge = 5000000

# Costo de backorder por dia
costo_backorder_dia = 250000

# Costo exceso de inventario
costo_exceso_capacidad = 5000000

# Costo de no safety stock por día
costo_safety_stock = 50000

# Los transportes solo tienen sentido desde el periodo 3, es dificil tomar deciciones para el mismo día
periodo_administrativo = 3

# Asumimos qe todo despacho tarda 2 días desde el momento que se envía la carga hasta que esta disponible para el consumo en planta
lead_time = 2

## Lectura de dataframes

In [ ]:
# Leer el archivo de excel
productos_df = pd.read_excel(io=file, sheet_name='ingredientes')
plantas_df = pd.read_excel(io=file, sheet_name='plantas')
asignador = AsignadorCapacidad(file)
unidades_almacenamiento_df = asignador.obtener_unidades_almacenamiento()
safety_stock_df = pd.read_excel(io=file, sheet_name='safety_stock')
consumo_proyectado_df = pd.read_excel(io=file, sheet_name='consumo_proyectado')
transitos_puerto_df = pd.read_excel(io=file, sheet_name='tto_puerto')
# transitos_planta_df = pd.read_excel(io=file, sheet_name='tto_plantas')
inventario_puerto_df = pd.read_excel(io=file, sheet_name='inventario_puerto')
costos_almacenamiento_df = pd.read_excel(io=file, sheet_name='costos_almacenamiento_cargas')
operaciones_portuarias_df = pd.read_excel(io=file, sheet_name='costos_operacion_portuaria')
operaciones_portuarias_df = operaciones_portuarias_df.set_index(['tipo_operacion', 'operador', 'puerto', 'ingrediente'])
fletes_df = pd.read_excel(io=file, sheet_name='fletes_cop_per_kg')
intercompany_df = pd.read_excel(io=file, sheet_name='venta_entre_empresas')

## Creacion de parametros del problema

### Tiempo

In [ ]:
# Obtener el conjunto de periodos
fechas = [datetime.strptime(x, '%d/%m/%Y') for x in consumo_proyectado_df.drop(columns=['planta', 'ingrediente']).columns]

periodos = [int(x.strftime('%Y%m%d')) for x in fechas]

periodo_anterior = fechas[0] - timedelta(days=1)
periodo_anterior = int(periodo_anterior.strftime('%Y%m%d'))

print(periodo_anterior,  periodos[0],periodos[-1])

### Productos

In [ ]:
productos = [productos_df.loc[i]['nombre'] for i in productos_df.index]

### Plantas

#### Tiempo de descarge de materiales

In [ ]:
# Generar plantas
plantas = dict()

for j in plantas_df.index:
    planta = plantas_df.loc[j]['planta']
    empresa = plantas_df.loc[j]['empresa']
    operacion_minutos = plantas_df.loc[j]['operacion_minutos']*plantas_df.loc[j]['plataformas']
    plantas[planta] = dict()
    plantas[planta]['empresa'] = empresa
    plantas[planta]['tiempo_total'] = operacion_minutos
    plantas[planta]['tiempo_ingrediente'] = dict()
    plantas[planta]['llegadas_puerto'] = dict()

    for p in productos:
        t_ingrediente = plantas_df.loc[j][p]
        plantas[planta]['tiempo_ingrediente'][p] = t_ingrediente
        plantas[planta]['llegadas_puerto'][p] = {t:list() for t in periodos}

#### Inventario en Planta

In [ ]:
unidades_almacenamiento_df['capacidad'] = unidades_almacenamiento_df.apply(lambda x: x[x['ingrediente_actual']] ,axis=1)
unidades_almacenamiento_df.drop(columns=productos, inplace=True)
unidades_almacenamiento_df = unidades_almacenamiento_df.groupby(['planta', 'ingrediente_actual'])[['cantidad_actual', 'capacidad']].sum().reset_index()

# Agregando la informacion de safety stock
unidades_almacenamiento_df = pd.merge(left=unidades_almacenamiento_df,
                                      right=safety_stock_df,
                                      left_on=['planta', 'ingrediente_actual'],
                                      right_on=['planta', 'ingrediente'],
                                      how='left').drop(columns='ingrediente')

In [ ]:
# Generar un diccionario para renombrar las columnas de tiempo en consumo proyectado
consumo_proyectado_renamer = {x: datetime.strptime(x, '%d/%m/%Y').strftime(
    '%Y%m%d') for x in consumo_proyectado_df.drop(columns=['planta', 'ingrediente']).columns}
# Efectuar el cambio de nombre
consumo_proyectado_df.rename(columns=consumo_proyectado_renamer, inplace=True)
# Unir con el consumo proyectado
unidades_almacenamiento_df = pd.merge(left=unidades_almacenamiento_df,
                                      right=consumo_proyectado_df,
                                      left_on=['planta', 'ingrediente_actual'],
                                      right_on=['planta', 'ingrediente'],
                                      how='left').drop(columns=['ingrediente']).rename(columns={'ingrediente_actual': 'ingrediente', 'cantidad_actual': 'cantidad'}).fillna(0.0)

print(unidades_almacenamiento_df.shape)
unidades_almacenamiento_df.head()
# Llenar la informacion de los inventarios
for i in unidades_almacenamiento_df.index:
    planta = unidades_almacenamiento_df.loc[i]['planta']
    ingrediente = unidades_almacenamiento_df.loc[i]['ingrediente']
    cantidad_inicial = unidades_almacenamiento_df.loc[i]['cantidad']
    capacidad_almacenamiento = unidades_almacenamiento_df.loc[i]['capacidad']
    safety_stock_dias = unidades_almacenamiento_df.loc[i]['dias_ss']

    if not 'inventarios' in plantas[planta].keys():
        plantas[planta]['inventarios'] = dict()

    if not ingrediente in plantas[planta]['inventarios'].keys():
        plantas[planta]['inventarios'][ingrediente] = dict()

    if not 'capacidad' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['capacidad'] = capacidad_almacenamiento

    if not 'inventario_final' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['inventario_final'] = dict()

    if not 'llegadas' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['llegadas'] = dict()

    if not 'consumo' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['consumo'] = dict()

    if not 'backorder' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['backorder'] = dict()

    if not 'safety_stock' in plantas[planta]['inventarios'][ingrediente].keys():
        plantas[planta]['inventarios'][ingrediente]['safety_stock'] = dict()

    #if not 'exceso_capacidad' in plantas[planta]['inventarios'][ingrediente].keys():
    #    plantas[planta]['inventarios'][ingrediente]['exceso_capacidad'] = dict()

    plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo_anterior] = cantidad_inicial

    consumo_total = sum([unidades_almacenamiento_df.loc[i][str(t)]
                        for t in periodos])

    if consumo_total > 0:

        plantas[planta]['inventarios'][ingrediente]['safety_stock_dias'] = safety_stock_dias

        safety_stock_kg = consumo_total*safety_stock_dias/len(periodos)

        plantas[planta]['inventarios'][ingrediente]['safety_stock_kg'] = safety_stock_kg

        for periodo in periodos:
            # Agregar las variables de inventario
            inventario_var_name = f'I_{planta}_{ingrediente}_{periodo}'
            inventario_var = pu.LpVariable(
                name=inventario_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo] = inventario_var

            """
            # Agregar las variables de exceso de inventario
            exceso_capacidad_var_name = f'M_{planta}_{ingrediente}_{periodo}'
            exceso_capacidad_var = pu.LpVariable(
                name=exceso_capacidad_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['exceso_capacidad'][periodo] = exceso_capacidad_var
            """
            
            # Agregar las listas a donde llegarán los transportes
            plantas[planta]['inventarios'][ingrediente]['llegadas'][periodo] = list()

            # Agregar las variables de backorder
            bak_var_name = f'B_{planta}_{ingrediente}_{periodo}'
            bak_var = pu.LpVariable(
                name=bak_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['backorder'][periodo] = bak_var

            # Agregar las variables de Safety Stock
            if capacidad_almacenamiento > safety_stock_kg + 2*cap_camion:
                ss_var_name = f'S_{planta}_{ingrediente}_{periodo}'
                ss_var = pu.LpVariable(
                    name=ss_var_name, lowBound=0.0, cat=pu.LpContinuous)
                plantas[planta]['inventarios'][ingrediente]['safety_stock'][periodo] = ss_var

            # Agregar el consumo proyectado
            consumo = unidades_almacenamiento_df.loc[i][str(periodo)]
            plantas[planta]['inventarios'][ingrediente]['consumo'][periodo] = consumo
    else:
        for periodo in periodos:
            # Dejar el inventario en el estado actual
            plantas[planta]['inventarios'][ingrediente]['inventario_final'][periodo] = cantidad_inicial

            # Agregar el consumo proyectado
            consumo = unidades_almacenamiento_df.loc[i][str(periodo)]
            plantas[planta]['inventarios'][ingrediente]['consumo'][periodo] = consumo

            """
            # Agregar las variables de exceso de inventario
            exceso_capacidad_var_name = f'M_{planta}_{ingrediente}_{periodo}'
            exceso_capacidad_var = pu.LpVariable(
                name=exceso_capacidad_var_name, lowBound=0.0, cat=pu.LpContinuous)
            plantas[planta]['inventarios'][ingrediente]['exceso_capacidad'][periodo] = exceso_capacidad_var
            """

#### Llegadas programadas anteriormente a Planta

In [ ]:
"""
for i in transitos_planta_df.index:
    planta = transitos_planta_df.loc[i]['planta']
    ingrediente = transitos_planta_df.loc[i]['ingrediente']
    cantidad = transitos_planta_df.loc[i]['cantidad']
    fecha = transitos_planta_df.loc[i]['fecha_llegada']
    periodo = int(fecha.strftime('%Y%m%d'))
    plantas[planta]['inventarios'][ingrediente]['llegadas'][periodo].append(0.0)
"""

### Cargas en Puerto

#### Crear cargas a partir de información de los transitos

In [ ]:
# Generar Cargas
cargas = dict()

# A partir de los transitos
for i in transitos_puerto_df.index:
    importacion = str(transitos_puerto_df.loc[i]['importacion']).replace(' ','')
    empresa = transitos_puerto_df.loc[i]['empresa']
    operador = transitos_puerto_df.loc[i]['operador']
    puerto = transitos_puerto_df.loc[i]['puerto']
    ingrediente = transitos_puerto_df.loc[i]['ingrediente']
    cantidad_kg = transitos_puerto_df.loc[i]['cantidad_kg']
    valor_cif = transitos_puerto_df.loc[i]['valor_kg']
    fecha = transitos_puerto_df.loc[i]['fecha_llegada']
    if not importacion in cargas.keys():
        cargas[importacion] = dict()
    
    cargas[importacion]['empresa'] = empresa
    cargas[importacion]['operador'] = operador
    cargas[importacion]['puerto'] = puerto
    cargas[importacion]['ingrediente'] = ingrediente
    cargas[importacion]['valor_cif'] = valor_cif
    cargas[importacion]['inventario_inicial'] = 0
    cargas[importacion]['costo_almacenamiento'] = {int(t.strftime('%Y%m%d')):0 for t in fechas}
    cargas[importacion]['llegadas'] = dict()
    cargas[importacion]['fecha_inicial'] = int(fecha.strftime('%Y%m%d'))
    
    # Poner llegadas de materia
    while cantidad_kg > cap_descarge:
        cargas[importacion]['llegadas'][int(fecha.strftime('%Y%m%d'))] = cap_descarge
        cantidad_kg -= cap_descarge
        fecha = fecha + timedelta(days=1)
    
    if cantidad_kg > 0:
        cargas[importacion]['llegadas'][int(fecha.strftime('%Y%m%d'))] = cantidad_kg
    cargas[importacion]['fecha_final'] = int(fecha.strftime('%Y%m%d'))

    # Agregar las variables de inventario
    cargas[importacion]['inventario_al_final'] = dict()
    for t in periodos:
        var_name = f"O_{importacion}_{t}"
        lp_var = pu.LpVariable(name=var_name,
                               lowBound=0.0,
                               upBound=transitos_puerto_df.loc[i]['cantidad_kg'],
                               cat=pu.LpContinuous)
        cargas[importacion]['inventario_al_final'][t]=lp_var


#### Crear cargas a partir de invenatrios en puerto

In [ ]:
    
# A Partir de los inventarios en puerto    
for i in inventario_puerto_df.index:
    empresa = inventario_puerto_df.loc[i]['empresa']
    operador = inventario_puerto_df.loc[i]['operador']
    puerto = inventario_puerto_df.loc[i]['puerto']
    ingrediente = inventario_puerto_df.loc[i]['ingrediente']
    importacion = str(inventario_puerto_df.loc[i]['importacion']).replace(' ','')
    inventario_inicial = inventario_puerto_df.loc[i]['cantidad_kg']
    valor_cif = inventario_puerto_df.loc[i]['valor_cif_kg']
    fecha = inventario_puerto_df.loc[i]['fecha_llegada']

    if not importacion in cargas.keys():
        cargas[importacion] = dict()
    
    cargas[importacion]['empresa'] = empresa
    cargas[importacion]['operador'] = operador
    cargas[importacion]['puerto'] = puerto
    cargas[importacion]['ingrediente'] = ingrediente
    cargas[importacion]['valor_cif'] = valor_cif
    cargas[importacion]['inventario_inicial'] = inventario_inicial
    cargas[importacion]['costo_almacenamiento'] = {int(t.strftime('%Y%m%d')):0 for t in fechas}

    # Poner llegadas de materia
    cargas[importacion]['llegadas'] = {t.strftime('%Y%m%d'):0 for t in fechas}
    
    cargas[importacion]['fecha_inicial'] = int(fecha.strftime('%Y%m%d'))
    cargas[importacion]['fecha_final'] = int(fecha.strftime('%Y%m%d'))
    # Agregar las variables de inventario
    cargas[importacion]['inventario_al_final'] = dict()

    for t in periodos:

        var_name = f"O_{importacion}_{t}"
        lp_var = pu.LpVariable(name=var_name,
                               lowBound=0.0,
                               upBound=inventario_puerto_df.loc[i]['cantidad_kg'],
                               cat=pu.LpContinuous)
        cargas[importacion]['inventario_al_final'][t]=lp_var

#### Costos de almacenamiento

In [ ]:
# Agregar costos de almacenamiento a cada carga
for i in costos_almacenamiento_df.index:
    importacion = str(costos_almacenamiento_df.loc[i]['importacion']).replace(' ','')
    fecha = int(costos_almacenamiento_df.loc[i]['fecha_corte'].strftime('%Y%m%d'))
    valor_kg = costos_almacenamiento_df.loc[i]['valor_kg']

    if importacion in cargas.keys():
        if fecha in cargas[importacion]['costo_almacenamiento']:
            cargas[importacion]['costo_almacenamiento'][fecha] += valor_kg

#### Costos de Bodegaje

In [ ]:
# Agregar costos de bodegaje cuando es un producto en tránsito a puerto a cada carga
for importacion, carga in cargas.items():
    index = ('bodega', carga['operador'], carga['puerto'], carga['ingrediente'])
    valor_kg = operaciones_portuarias_df.loc[index]['valor_kg']
    if carga['fecha_inicial'] >= int(fechas[0].strftime('%Y%m%d')) and carga['fecha_final'] <= int(fechas[-1].strftime('%Y%m%d')):
        carga['costo_almacenamiento'][carga['fecha_final']] += valor_kg

#### Costos intercompany

In [ ]:
intercompany_df = intercompany_df.melt(id_vars='origen',
                                       value_vars=['contegral', 'finca'],
                                       var_name='destino',
                                       value_name='intercompany')

intercompany_df.set_index(['origen', 'destino'], inplace=True)

#### Costos de transporte (fletes)

In [ ]:
# Encontrar el costo total de transporte por kilogramo
fletes_df = fletes_df.melt(id_vars=['puerto', 'operador', 'ingrediente'],
                           value_vars=list(plantas.keys()),
                           value_name='costo_per_kg',
                           var_name='planta')

# Calcular valor del flete
fletes_df['flete'] = cap_camion*fletes_df['costo_per_kg']

fletes_df = pd.merge(left=fletes_df,
                     right=plantas_df[['planta', 'empresa']],
                     left_on='planta',
                     right_on='planta')

fletes_df.set_index(['puerto', 'operador', 'ingrediente', 'planta'], inplace=True)

*Nota:* evitar crear variables de transporte si la capacidad de almacenamiento de la planta es 0 o e consumo es 0 

In [ ]:
# Informacion de transporte
for importacion, carga in cargas.items():
    puerto = carga['puerto']
    operador = carga['operador']
    ingrediente = carga['ingrediente']
    costo_envio = dict() 

    for nombre_planta, planta in plantas.items():
        empresa_destino = planta['empresa']
        costo_intercompany = intercompany_df.loc[(carga['empresa'], empresa_destino)]['intercompany']
        valor_intercompany = cap_camion*carga['valor_cif']*(costo_intercompany)
        flete = fletes_df.loc[(puerto,operador,ingrediente,nombre_planta)]['flete']
        valor_despacho_directo_kg = cap_camion*operaciones_portuarias_df.loc[('directo', operador, puerto, ingrediente)]['valor_kg']

        # Costo de flete
        costo_envio[nombre_planta] = dict()
        costo_envio[nombre_planta]['intercompany'] = costo_intercompany
        costo_envio[nombre_planta]['flete'] = flete
        costo_envio[nombre_planta]['cantidad_despacho'] = cap_camion
        costo_envio[nombre_planta]['valor_intercompany'] = valor_intercompany
        costo_envio[nombre_planta]['costo_despacho_directo'] = valor_despacho_directo_kg

        costo_envio[nombre_planta]['costo_envio'] = dict()
        costo_envio[nombre_planta]['tipo_envio'] = dict()
        costo_envio[nombre_planta]['variable_despacho'] = dict()

        # Tomar en cuenta solo los periodos relevantes
        periodo_final = periodos.index(periodos[-1])-lead_time+1

        for periodo in periodos[periodo_administrativo:periodo_final]:
            # Si el periodo esta entre la fecha de llegada, colocar operacion portuaria por despacho directo.
            if periodo >= carga['fecha_inicial'] and periodo <= carga['fecha_final']:
                costo_envio[nombre_planta]['costo_envio'][periodo] = valor_intercompany + flete + valor_despacho_directo_kg
                costo_envio[nombre_planta]['tipo_envio'][periodo] = 'directo'
                
            else:
                costo_envio[nombre_planta]['costo_envio'][periodo] = valor_intercompany + flete
                costo_envio[nombre_planta]['tipo_envio'][periodo] = 'indirecto'

            # Variable de transporte
                
            # Antes de crear las variables de transporte, es importante saber si la planta tiene consumo del ingrediente
            if ingrediente in planta['inventarios'].keys():            

                consumo_total = sum([c for p,c in planta['inventarios'][ingrediente]['consumo'].items()])

                if consumo_total > 0:

                    transporte_var_name = f'T_{importacion}_{nombre_planta}_{periodo}'
                    transporte_var = pu.LpVariable(name=transporte_var_name,
                                        lowBound=0,
                                        cat=pu.LpInteger)
                    
                    costo_envio[nombre_planta]['variable_despacho'][periodo] = transporte_var

                    # Colocar la variable en la planta dos periodos despues
                    periodo_llegada = periodos[periodos.index(periodo)+lead_time]
                    plantas[nombre_planta]['inventarios'][ingrediente]['llegadas'][periodo_llegada].append(transporte_var)           


        carga['costo_despacho'] = costo_envio
        

# Modelo matemático

## Sets:

$t$: periodos con $t \in T$

$p$: productos con $p \in P$

$i$: cargas con $i \in T$

$j$: plantas con $j \in J$


## Parametros:

$CB:$ Costo de backorder por día

$CT_{ij}$: Costo de transportar la carga $i$ hacia la planta $j$

$CA_{it}$: Costo de mantener la carga $i$ almacenada al final del periodo $p$

$AR_{it}$: Cantidad de producto llegando a la carga $i$ durante el periodo $p$

$DM_{pjt}$: Demanda del producto $p$ en la planta $j$ durante el periodo $t$

$CP_{pjt}$: Capacidad de almacenar el producto $p$ en la planta $j$

$IP_{i}$: Inventario inicial de la carga $i$

$TJ_{pjt}$: Cantidad programada del producto $p$ llegando a la planta $j$ durante el periodo $t$ 

$IJ_{pj}$: Inventario inicial de producto $p$ en la planta $j$ 

$MI_{pjt}$: Inventario mínimo a mantener del producto $p$ en la planta $j$ al final del periodo $t$

$MX_{pjt}$: Inventario máximo a mantener del producto $p$ en la planta $j$ al final del periodo $t$


## Diccionarios

Carga[i][Producto]: Producto que contiene la carga i

Carga[i][Inventario_Inicial]: Inventario Inicial de la carga i
 


## Variables:

$T_{ijt}$: Variable entera, Cantidad de camiones de 34 Toneladas a despachar de la carga $i$ hasta la planta $j$ durante el periodo $t$

$O_{it}$: Continua, Cantidad de toneladas de la carga $i$ almacenadas al final del periodo $t$

$I_{pjt}$: Cantidad del producto $p$ almacenado en la planta $j$ al final del periodo $t$

$B_{pjt}$: Cantidad del producto $p$ de backorder en la planta $j$ al final del periodo $t$

$S_{pjt}$: Cantidad del producto $p$ por debajo del SS en la planta $j$ al final del periodo $t$

$M_{pjt}$: Unidades de exceso del producto $p$ en la planta $j$ al final del periodo $t$ sobre el inventario objetivo

$X_{pjt}$: Unidades por debajo del producto $p$ en la planta $j$ al final del periodo $t$ bajo el inventario objetivo



## Funcion Objetivo:

$$min \sum_{i}\sum_{j}\sum_{t}{CT_{ijt}*T_{ijt}} + \sum_{i}\sum_{t}CA_{it}O_{it} + \sum_{pjt}{CB*B}  + PP \sum_{M}\sum_{P}\sum_{J}{M_{mpj}} + PP \sum_{M}\sum_{P}\sum_{J}{X_{mpj}}$$

In [ ]:
funcion_objetivo = list()

### Costo por transporte

El costo del transporte esta dado por el producto escalar entre los costos de envio, que ya incluyen fletes, costos porturarios y costos intercompany

$$\sum_{i}\sum_{j}\sum_{t}{CT_{ijt}*T_{ijt}}$$

In [ ]:
for periodo in periodos:
    for impo, carga in cargas.items():
        for nombre_planta, planta in plantas.items():
            if periodo in carga['costo_despacho'][nombre_planta]['variable_despacho'].keys():
                # CT_ijt*T_ijt
                costo_envio = carga['costo_despacho'][nombre_planta]['costo_envio'][periodo]# + periodos.index(periodo)
                var_envio = carga['costo_despacho'][nombre_planta]['variable_despacho'][periodo]
                funcion_objetivo.append(costo_envio*var_envio)


### Costo por almacenamiento en puerto

El costo por almacenamiento esta dado por el producto escalar entre los costos de almacenamiento que incluyen el costo el costo de operacion portuaria de llevar el material desde el barco hasta la bodega y, la tarifa por almacenamiento que se paga periódicamente luego de los días libres. 

$$\sum_{i}\sum_{t}CA_{it}O_{it}$$

In [ ]:
for periodo in periodos:
    for impo, carga in cargas.items():
        costo_almaenamiento = carga['costo_almacenamiento'][periodo]
        inventario_al_final = carga['inventario_al_final'][periodo]
        if costo_almaenamiento > 0:
            funcion_objetivo.append(costo_almaenamiento*inventario_al_final)


### Costo de Backorder

El costo por backorder es una penalización a la función objetivo, donde se carga un valor determinado por cada kilogramo de material que no esté disponible para el consumo

$\sum_{pjt}{CB*B}$

In [ ]:
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        for periodo, var in ingrediente['backorder'].items():
            funcion_objetivo.append(costo_backorder_dia*var)
        

### Costo por no alcanzar el inventario de seguridad (pendiente)

In [ ]:
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        if 'safety_stock' in planta['inventarios'][nombre_ingrediente].keys():
            for periodo, var in ingrediente['safety_stock'].items():
                funcion_objetivo.append(costo_safety_stock*var)

### Costo por exceder capacidad de almacenamiento

In [ ]:
"""
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        for periodo, var in ingrediente['exceso_capacidad'].items():
            funcion_objetivo.append(costo_exceso_capacidad*var)
"""

## Restricciones:


### Balance de masa en cargas

El inventario al final del periodo es igual a:

- el inventario al final del periodo anterior;
- más las llegadas planeadas;
- menos los despachos hacia plantas

$$ O_{it} =  O_{i(t-1)} + AR_{it} - 34000\sum_{J}{T_{ijt}} \hspace{1cm} \forall i \in I, t \in T$$

In [ ]:
rest_balance_masa_puerto = list()

for importacion, carga in cargas.items():
    for periodo in periodos:

        left = list()
        right = list()
        
        # Oit
        Oit = carga['inventario_al_final'][periodo]
        left.append(Oit)

        # Oi(t-1)
        if periodo == periodos[0]:
            Oitant = carga['inventario_inicial']
        else:
            t_anterior = periodos[periodos.index(periodo)-1]
            Oitant = carga['inventario_al_final'][t_anterior]
        right.append(Oitant)

        # ARit
        if periodo in carga['llegadas'].keys():
            ar = carga['llegadas'][periodo]
            right.append(ar)

        # - 34000*Sum(Tijt)
        for planta, despacho in carga['costo_despacho'].items():            
            if periodo in despacho['variable_despacho'].keys():
                var_despacho = despacho['variable_despacho'][periodo]
                left.append(cap_camion*var_despacho)
        
        
        name = f'balance_masa_{importacion}_al_final_de_{periodo}'
        rest = (pu.lpSum(left) == pu.lpSum(right), name)
        
        rest_balance_masa_puerto.append(rest)


### Balance de masa en plantas

El inventario en planta al final del periodo es igual a:

- el inventario al final del periodo anterior;
- más las llegadas ya programadas;
- más las llegadas planeadas;
- menos la demanda
- más el backorder, que compensa cuando el inventario más las llegadas no son suficientes

$$ I_{pjt} = I_{pj(t-1)} + TJ_{pjt} + \sum_{i}{T_{ij(t-2)}} -  DM_{pjt} + B_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$


In [ ]:
rest_balance_masa_planta =  list()
for nombre_planta, planta in plantas.items():
    for nombre_ingrediente, ingrediente in planta['inventarios'].items():
        for periodo in periodos:

            left = list()
            right = list()

            # Ipjt
            Spjt = ingrediente['inventario_final'][periodo]
            left.append(Spjt)

            # Ipj(t-1)
            if periodo == periodos[0]:
                Ipj_tanterior = ingrediente['inventario_final'][periodo_anterior]
            else:
                p_anterior = periodos[periodos.index(periodo)-1]
                Ipj_tanterior = ingrediente['inventario_final'][p_anterior]
                
            right.append(Ipj_tanterior)
                
            # + TJ


            # + Tijt
            if periodo in ingrediente['llegadas'].keys():
                for llegada_planeada_var in ingrediente['llegadas'][periodo]:
                    if type(llegada_planeada_var) == pu.LpVariable:
                        right.append(cap_camion*llegada_planeada_var)
                    else:
                        right.append(llegada_planeada_var)

            # - DMpjt
           
            if periodo in ingrediente['consumo'].keys():
                DMpjt = ingrediente['consumo'][periodo]
                left.append(DMpjt)
            
            # + Baclorder
            if periodo in ingrediente['backorder'].keys():
                bak_var = ingrediente['backorder'][periodo]
                right.append(bak_var)

            name = f'balance_planta_{nombre_planta}_de_{nombre_ingrediente}_al_final_de_{periodo}'
            rest = (pu.lpSum(left) == pu.lpSum(right), name)

            rest_balance_masa_planta.append(rest)

### Capacidad de recepción por planta

En una planta y un periodo en particular, la suma del producto entre el tiempo del ingrediente y la cantidad de camiones llegando no debe superar el tiempo total disponible en la planta

$$ \sum_{I}{TiempoIngrediente_{pj}*T_{ijt}} \leq TiempoTotal_{t} \hspace{1cm} \forall p \in P, t \in T$$

In [ ]:
rest_llegada_material=list()
for nombre_planta, planta in plantas.items():
    tiempo_total = planta['tiempo_total']
    for periodo in periodos:
        left_expresion = list()
        for ingrediente, parametros in planta['inventarios'].items():
            tiempo_ingrediente = planta['tiempo_ingrediente'][ingrediente]
            if periodo in parametros['llegadas'].keys():
                for var_llegada in parametros['llegadas'][periodo]:
                    left_expresion.append(tiempo_ingrediente*var_llegada)
        
        # omitir restricciones sin expresiones al lado izquiero
        if len(left_expresion)>0:
            rest_name = f'Llegada_material_{nombre_planta}_durante_{periodo}'
            rest = (pu.lpSum(left_expresion)<=tiempo_total, rest_name)
            rest_llegada_material.append(rest)
        

### Capacidad de almacenamiento

$$ I_{pjt} \leq CP_{pj} + M_{pjt} \hspace{1cm} \forall p \in P, t \in T$$

In [ ]:
"""
rest_capacidad_almacenamiento = list()
for nombre_planta, planta in plantas.items():
    for ingrediente, inventarios in planta['inventarios'].items(): 
        CPpj = inventarios['capacidad']
        for periodo, inventario_final_var in inventarios['inventario_final'].items():
            if type(inventario_final_var) == pu.LpVariable:
                rest_name = f'capacidad_almacenamiento_{nombre_planta}_de_{ingrediente}_en_{periodo}'
                #if periodo in inventarios['exceso_capacidad'].keys():
                Mpjt = inventarios['exceso_capacidad'][periodo]
                rest = (inventario_final_var <= CPpj + Mpjt, rest_name)
                #else: 
               #     rest = (inventario_final_var <= CPpj, rest_name)
                rest_capacidad_almacenamiento.append(rest)
"""

### No superar el inventario máximo

El inventario al final de un día cualquiera debe estar bajo el nivel máximo, por lo que penalizaremos en la función objetivo una variable de holgura para tal efecto
$$ I_{pjt} \leq MX_{pjt} + M_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$

### Superar el inventario de seguridad

El inventario al final de un día cualquiera debe estar bajo el nivel máximo, por lo que penalizaremos en la función objetivo una variable de holgura para tal efecto
$$ I_{pjt} \geq MX_{pjt} + M_{pjt} \hspace{1cm} \forall p \in P, j \in J, t \in T$$

In [ ]:
rest_safety_stock = list()
for nombre_planta, planta in plantas.items():
    for ingrediente, inventarios in planta['inventarios'].items(): 
        if 'safety_stock_kg' in inventarios.keys():
            SS = inventarios['safety_stock_kg']
            #for periodo, variable in inventarios['inventario_final'].items():
            if len(inventarios['safety_stock'].keys())>0:
                for periodo in periodos:                  
                    rest_name = f'safety_stock_en_{nombre_planta}_de_{ingrediente}_durante_{periodo}'      
                    Ipjt = inventarios['inventario_final'][periodo]
                    Spij = inventarios['safety_stock'][periodo]
                    
                    rest = (Ipjt +  Spij >= SS, rest_name)
                    rest_safety_stock.append(rest)

# Resolviendo el modelo
## Generando modelo

In [ ]:
problema = pu.LpProblem(name='Bios_Solver', sense=pu.LpMinimize)

# Agregando funcion objetivo
problema+= pu.lpSum(funcion_objetivo)

# Agregando balance de masa puerto
for rest in rest_balance_masa_puerto:
    problema+=rest

# Agregando balande ce masa en planta
for rest in rest_balance_masa_planta:
    problema+=rest
    
# Agregando capacidad de recepcion
for rest in rest_llegada_material:
    problema+=rest

# Agregando capacidad de almacenamiento
#for rest in rest_capacidad_almacenamiento:
#    problema+=rest

# Agregando inventario de seguridad
for rest in rest_safety_stock:
    problema+=rest

    

## Ejecutando modelo

In [ ]:
t_limit_minutes = 60*12
cpu_count = max(1, os.cpu_count()-1)
gap = 5000000
print('cpu count', cpu_count)
engine = pu.PULP_CBC_CMD(
            timeLimit=60*t_limit_minutes,
            gapAbs=gap,
            warmStart=False,
            threads=cpu_count)
problema.solve(solver=engine)

# Construccion de reporte

In [ ]:
reporte_puerto_dict=dict()
reporte_transporte_dict=dict()
reporte_plantas_dict=dict()

## Reporte de puerto

In [ ]:
reporte_puerto_dict['importacion'] = list()
reporte_puerto_dict['empresa'] = list()
reporte_puerto_dict['operador'] = list()
reporte_puerto_dict['puerto'] = list()
reporte_puerto_dict['ingrediente'] = list()
reporte_puerto_dict['valor_cif'] = list()
reporte_puerto_dict['periodo'] = list()
reporte_puerto_dict['costo_almacenamiento'] = list()
reporte_puerto_dict['llegadas'] = list()
reporte_puerto_dict['inventario'] = list()

for importacion, carga in cargas.items():

    reporte_puerto_dict['importacion'].append(importacion)
    reporte_puerto_dict['empresa'].append(carga['empresa'])
    reporte_puerto_dict['operador'].append(carga['operador'])
    reporte_puerto_dict['puerto'].append(carga['puerto'])
    reporte_puerto_dict['ingrediente'].append(carga['ingrediente'])
    reporte_puerto_dict['valor_cif'].append(carga['valor_cif'])
    reporte_puerto_dict['periodo'].append(periodo_anterior)
    reporte_puerto_dict['costo_almacenamiento'].append(0.0)
    reporte_puerto_dict['llegadas'].append(0)
    reporte_puerto_dict['inventario'].append(carga['inventario_inicial'])
    

    for periodo in periodos:
        reporte_puerto_dict['importacion'].append(importacion)
        reporte_puerto_dict['empresa'].append(carga['empresa'])
        reporte_puerto_dict['operador'].append(carga['operador'])
        reporte_puerto_dict['puerto'].append(carga['puerto'])
        reporte_puerto_dict['ingrediente'].append(carga['ingrediente'])
        reporte_puerto_dict['valor_cif'].append(carga['valor_cif'])
        reporte_puerto_dict['periodo'].append(periodo)
        reporte_puerto_dict['costo_almacenamiento'].append(carga['costo_almacenamiento'][periodo])
        if periodo in carga['llegadas'].keys():
            reporte_puerto_dict['llegadas'].append(carga['llegadas'][periodo])
        else:
            reporte_puerto_dict['llegadas'].append(0.0)
        reporte_puerto_dict['inventario'].append(cargas[importacion]['inventario_al_final'][periodo].varValue)

reporte_puerto_df = pd.DataFrame(reporte_puerto_dict)
reporte_puerto_df['costo_total_almacenamiento'] = reporte_puerto_df['inventario']*reporte_puerto_df['costo_almacenamiento']


## Reporte transporte

In [ ]:
reporte_transporte_dict['importacion'] = list()
reporte_transporte_dict['empresa'] = list()
reporte_transporte_dict['operador'] = list()
reporte_transporte_dict['puerto'] = list()
reporte_transporte_dict['ingrediente'] = list()
reporte_transporte_dict['periodo'] = list()
reporte_transporte_dict['tipo'] = list()
reporte_transporte_dict['planta'] = list()
reporte_transporte_dict['intercompany'] = list()
reporte_transporte_dict['costo_intercompany'] = list()
reporte_transporte_dict['flete'] = list()
reporte_transporte_dict['cantidad_despacho_por_camion'] = list()
reporte_transporte_dict['costo_portuario_despacho_directo'] = list()
reporte_transporte_dict['cantidad_despacho'] = list()
reporte_transporte_dict['cantidad_camiones_despachados'] = list()
reporte_transporte_dict['cantidad_despachada'] = list()
reporte_transporte_dict['costo_por_camion'] = list()

for importacion, carga in cargas.items():
    for nombre_planta, despacho in carga['costo_despacho'].items():
        for periodo in periodos:
            if periodo in despacho['variable_despacho'].keys():
                # if despacho['variable_despacho'][periodo].varValue > 0:

                reporte_transporte_dict['importacion'].append(importacion)
                reporte_transporte_dict['empresa'].append(carga['empresa'])
                reporte_transporte_dict['operador'].append(carga['operador'])
                reporte_transporte_dict['puerto'].append(carga['puerto'])
                reporte_transporte_dict['ingrediente'].append(carga['ingrediente'])
                reporte_transporte_dict['periodo'].append(periodo)
                reporte_transporte_dict['tipo'].append(despacho['tipo_envio'][periodo])
                reporte_transporte_dict['planta'].append(nombre_planta)
                reporte_transporte_dict['intercompany'].append(despacho['intercompany'])
                reporte_transporte_dict['costo_intercompany'].append(despacho['valor_intercompany'])
                reporte_transporte_dict['flete'].append(despacho['flete'])
                reporte_transporte_dict['cantidad_despacho_por_camion'].append(despacho['cantidad_despacho'])
                reporte_transporte_dict['costo_portuario_despacho_directo'].append(despacho['costo_despacho_directo'])
                reporte_transporte_dict['costo_por_camion'].append(despacho['costo_envio'][periodo])
                reporte_transporte_dict['cantidad_despacho'].append(cap_camion)
                reporte_transporte_dict['cantidad_camiones_despachados'].append(despacho['variable_despacho'][periodo].varValue)
                reporte_transporte_dict['cantidad_despachada'].append(cap_camion*despacho['variable_despacho'][periodo].varValue)

reporte_transporte_df = pd.DataFrame(reporte_transporte_dict)
reporte_transporte_df['costo_total_despacho'] = reporte_transporte_df['costo_por_camion']*reporte_transporte_df['cantidad_camiones_despachados']

## Reporte de Planta

In [ ]:
reporte_plantas_dict=dict()
reporte_plantas_dict['planta'] = list()
reporte_plantas_dict['empresa'] = list()
reporte_plantas_dict['ingrediente'] = list()
reporte_plantas_dict['periodo'] = list()
reporte_plantas_dict['capcidad'] = list()
reporte_plantas_dict['consumo'] = list()
reporte_plantas_dict['backorder'] = list()
reporte_plantas_dict['safety_stock_kg'] =  list()
reporte_plantas_dict['inventario_final'] = list()

for nombre_planta, planta in plantas.items():
    for ingrediente, inventario in planta['inventarios'].items():

        reporte_plantas_dict['planta'].append(nombre_planta)
        reporte_plantas_dict['empresa'].append(planta['empresa'])
        reporte_plantas_dict['ingrediente'].append(ingrediente)
        reporte_plantas_dict['periodo'].append(periodo_anterior)
        reporte_plantas_dict['capcidad'].append(inventario['capacidad'])
        reporte_plantas_dict['consumo'].append(0.0)
        reporte_plantas_dict['backorder'].append(0.0)
        reporte_plantas_dict['safety_stock_kg'].append(0.0)
        reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo_anterior])

        for periodo in periodos:
            reporte_plantas_dict['planta'].append(nombre_planta)
            reporte_plantas_dict['empresa'].append(planta['empresa'])
            reporte_plantas_dict['ingrediente'].append(ingrediente)
            reporte_plantas_dict['periodo'].append(periodo)
            reporte_plantas_dict['capcidad'].append(inventario['capacidad'])
            reporte_plantas_dict['consumo'].append(inventario['consumo'][periodo])
            if periodo in inventario['backorder'].keys():
                reporte_plantas_dict['backorder'].append(inventario['backorder'][periodo].varValue)
            else:
                reporte_plantas_dict['backorder'].append(0.0)

            if 'safety_stock_kg' in inventario.keys():
                reporte_plantas_dict['safety_stock_kg'].append(inventario['safety_stock_kg'])
            else:
                reporte_plantas_dict['safety_stock_kg'].append(0.0)
            
            if type(inventario['inventario_final'][periodo])==pu.pulp.LpVariable:
                reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo].varValue)
            else:
                reporte_plantas_dict['inventario_final'].append(inventario['inventario_final'][periodo])

reporte_plantas_df = pd.DataFrame(reporte_plantas_dict)

## Escribiendo el archivo

In [ ]:
with pd.ExcelWriter(path="archivo_schema_12_horas.xlsx") as writer:
    reporte_puerto_df.to_excel(writer, sheet_name='Puertos', index=False)
    reporte_plantas_df.to_excel(writer, sheet_name='Plantas', index=False)
    reporte_transporte_df.to_excel(writer, sheet_name='Despachos', index=False)